In [186]:
import pandas as pd
from pandas import Series, DataFrame
import json 
import os
import requests 
import math
import sqlite3
%matplotlib inline
import matplotlib
import copy
import matplotlib.pyplot as plt
from ast import literal_eval

matplotlib.rcParams.update({'font.size': 15})


In [187]:
#general budget dataplot from new data 
budget = pd.read_csv("newbudget1.csv")
budget


,Agency,Organization,Account Type,Character Code,Object,2015,2016,2017,2018
0,50 - LIBRARY,50100 - LIB PUBLIC SERVICE,Revenue,41 - GENERAL REVENUES,41110 - REAL ESTATE TAXES,"-11,355,535","-11,941,627","-12,603,779","-13,080,448"
1,50 - LIBRARY,50300 - LIB COL RES & ACCESS,Revenue,41 - GENERAL REVENUES,41110 - REAL ESTATE TAXES,"-2,056,164","-2,222,626","-2,199,357","-2,268,969"
2,50 - LIBRARY,50400 - LIB FACILITIES,Revenue,41 - GENERAL REVENUES,41110 - REAL ESTATE TAXES,"-1,766,869","-1,768,986","-1,963,404","-1,976,738"
3,51 - PARKS,51220 - PARK MTC FORESTRY GENERAL,Revenue,49 - TRANSFER IN,49124 - TRANSFER IN FROM OTHER RESTRIC,"-1,000,000","-2,500,000","-3,100,342",-
4,44 - STREETS,44220 - ST RECYCLING COLLECTION,Revenue,43 - CHARGES FOR SERVICES,43445 - SALE OF RECYCLABLES,"-841,013","-866,761","-1,218,608","-892,803"
5,65 - PLANNING,65300 - PL METROPOLITAN PLANNING ORG,Revenue,49 - TRANSFER IN,49123 - TRANSFER IN FROM GRANTS,"-683,795","-809,851",-,-
6,20 - MUNICIPAL COURT,20100 - MUNICIPAL COURT,Revenue,43 - CHARGES FOR SERVICES,43120 - COURT FEES,"-640,287","-624,341","-515,025","-421,900"
7,30 - FIRE,30252 - FIRE INSPECTION ELEVATOR,Revenue,44 - LICENSES AND PERMITS,44230 - ELEVATOR PERMITS AND INSPECTS,"-617,670","-606,447","-534,775","-555,937"
8,40 - ENGINEERING,40470 - ENG FAC MAINT ADMIN,Expense,58 - INTER DEPART BILLING,58131 - ID BILLING TO POLICE,"-531,289","-533,701","-536,303","-611,303"
9,50 - LIBRARY,50300751 - LIB COL RES & ACCESS CENTRAL,Revenue,43 - CHARGES FOR SERVICES,43568 - CATALOGING SERVICES,"-455,858","-454,255","-454,255","-454,290"


In [188]:
#code references for the new data
coderef = pd.read_csv("newbudget1codereferences.csv")
coderef


,Organization,Agency,Service,Subservice,Location
0,10110 - ASSESSOR RESIDENTIAL,10 - ASSESSOR,101 - ASSESSOR,1010 - RESIDENTIAL,0000 - UNDEFINED
1,10120 - ASSESSOR PERSONAL PROPERTY,10 - ASSESSOR,101 - ASSESSOR,1020 - PERSONAL PROPERTY,0000 - UNDEFINED
2,10130 - ASSESSOR COMMERCIAL,10 - ASSESSOR,101 - ASSESSOR,1030 - COMMERCIAL,0000 - UNDEFINED
3,10140 - ASSESSOR TAX ROLL PREPARATION,10 - ASSESSOR,101 - ASSESSOR,1040 - TAX ROLL PREPARATION,0000 - UNDEFINED
4,10150 - ASSESSOR BOARD OF REVIEW,10 - ASSESSOR,101 - ASSESSOR,1050 - BOARD OF REVIEW,0000 - UNDEFINED
5,1100 - BS GENERAL,00 - UNDEFINED,000 - UNDEFINED,0000 - UNDEFINED,0000 - UNDEFINED
6,110010 - GENERAL PURPOSE REVENUE,00 - UNDEFINED,000 - UNDEFINED,0000 - UNDEFINED,0000 - UNDEFINED
7,110020 - GENERAL FUND EXPENSE,00 - UNDEFINED,000 - UNDEFINED,0000 - UNDEFINED,0000 - UNDEFINED
8,11100 - ATTY ORDINANCE ENFORCEMENT,11 - ATTORNEY,111 - ORDINANCE ENFORCEMENT,0000 - UNDEFINED,0000 - UNDEFINED
9,11200 - ATTY LEGISLATIVE SERVICES,11 - ATTORNEY,112 - LEGISLATIVE SERVICES,0000 - UNDEFINED,0000 - UNDEFINED


In [239]:
#services provided per planning
planningservice = coderef[coderef["Agency"]== "65 -  PLANNING"].copy()
planningservice
#planning for each year
planningbudget = budget[budget["Agency"] == "65 -  PLANNING"].copy()
pb2015 = planningbudget["2015"]
pb2015 = pb2015[pb2015.notnull()]
pb2015 = pb2015[pb2015.replace(",","")] 
pb2015 = pb2015[pb2015.replace(" -  ", "0")]



ValueError: cannot reindex from a duplicate axis

In [190]:
#planning expense and revenue per year
years = ["2015", "2016","2017", "2018"]
newplanningservice = {}
counter = []
for organization in planningservice["Organization"]:
    for org in planningbudget["Organization"]:
        if organization == org:
            if counter.count(org) == 0:
                for year in years:
                    a = planningbudget[planningbudget["Organization"]==org]
                    #a[year] = planningbudget[planningbudget["organization"==org]][year]
                    newplanningservice.update(a)
                    counter.append(org)
new=pd.DataFrame(newplanningservice)   
#still in the works 

In [135]:
#coderef counts of planning 
planningservice

,Organization,Agency,Service,Subservice,Location
513,65101 - PL NEIGH PLAN PRES DESIGN ADMI,65 - PLANNING,651 - NEIGH PLANNING PRES AND DESIGN,6501 - ADMINISTRATION,0000 - UNDEFINED
514,65110 - PL NEIGHBORHOOD PLANNING,65 - PLANNING,651 - NEIGH PLANNING PRES AND DESIGN,6510 - NEIGHBORHOOD PLANNING,0000 - UNDEFINED
515,65111 - PL PRESERVATION PLANNING,65 - PLANNING,651 - NEIGH PLANNING PRES AND DESIGN,6511 - PRESERVATION PLANNING,0000 - UNDEFINED
516,65112 - PL MADISON ARTS PROGRAM,65 - PLANNING,651 - NEIGH PLANNING PRES AND DESIGN,6512 - MADISON ARTS PROGRAM,0000 - UNDEFINED
517,65113 - PL URBAN DESIGN COMMISSION,65 - PLANNING,651 - NEIGH PLANNING PRES AND DESIGN,6513 - URBAN DESIGN COMMISSION,0000 - UNDEFINED
518,65114 - PL SPECIAL PROJECTS,65 - PLANNING,651 - NEIGH PLANNING PRES AND DESIGN,6514 - SPECIAL PROJECTS,0000 - UNDEFINED
519,65115 - PL TIF CDA REDEVELOPMENT PLAN,65 - PLANNING,651 - NEIGH PLANNING PRES AND DESIGN,6515 - TIF CDA REDEVELOPMENT PLANNING,0000 - UNDEFINED
520,65201 - PL COMP PLAN AND DEVELOP ADMIN,65 - PLANNING,652 - COMP PLANNING AND DEV REVIEW,6501 - ADMINISTRATION,0000 - UNDEFINED
521,65220 - PL PLAN COMMISSION DEV REVIEW,65 - PLANNING,652 - COMP PLANNING AND DEV REVIEW,6520 - PLAN COMMISSION DEV REVIEW,0000 - UNDEFINED
522,65221 - PL DEVELP LAND USE ZONING REG,65 - PLANNING,652 - COMP PLANNING AND DEV REVIEW,6521 - DEVELP LAND USE AND ZONING REG,0000 - UNDEFINED


In [228]:
#data frame for planning yearly budget by charactercodes 
agencydic= {"2015": {}, "2016": {} , "2017": {} , "2018": {} }

for year in agencydic:
    for code in planningbudget["Character Code"]:
        if code not in agencydic[year]:
            a = planningbudget[planningbudget["Character Code"]==code]
            summ = 0
            for money in a[year].notnull():
                money=money.replace(",","")
                if type(money) == int:
                    summ = summ + money
            agencydic[year][code] = copy.deepcopy(summ)
adf = pd.DataFrame(agencydic["2015"], index=["2015"])
bdf = pd.DataFrame(agencydic["2016"], index=["2016"])
cdf = pd.DataFrame(agencydic["2017"], index=["2017"])
ddf = pd.DataFrame(agencydic["2018"], index=["2018"])
planyearsum = pd.concat([adf, bdf, cdf, ddf])
planyearsum


AttributeError: 'bool' object has no attribute 'replace'

In [ ]:
Africa = dff[dff["continent"]=="Africa"]["population"].mean()/1000000
Asia = dff[dff["continent"]=="Asia"]["population"].mean()/1000000
Australia = dff[dff["continent"]=="Australia"]["population"].mean()/1000000
CentralAmerica = dff[dff["continent"]=="Central America"]["population"].mean()/1000000
Europe = dff[dff["continent"]=="Europe"]["population"].mean()/1000000
NorthAmerica = dff[dff["continent"]=="North America"]["population"].mean()/1000000
SouthAmerica = dff[dff["continent"]=="South America"]["population"].mean()/1000000
tabless = pd.DataFrame({"Continent": ["Africa", "Asia", "Australia", "Central America", "Europe", "North America", "South America"], "avg_pop": [Africa, Asia, Australia, CentralAmerica, Europe, NorthAmerica, SouthAmerica]}) 
tabless = tabless.set_index('Continent')
ax = tabless.plot.bar()
ax.set_ylabel("avg population per continent (millions)")
ax.set_xlabel("")
ax